## Summary

---

## Imports

In [ ]:
from pathlib import Path

import pyarrow as pa
import pyarrow.parquet as pq
from tqdm.auto import tqdm

## Parameters

In [ ]:
NOTEBOOK_DIR = Path("31_run_protbert").resolve()
# NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

In [ ]:
DATASET_NAME = "cagi6-sherloc"
DATASET_PATH = str(
    NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
)

# MODEL_NOTEBOOK_NAME = "31_run_proteinsolver"
# MODEL_NOTEBOOK_NAME = "31_run_protbert"
MODEL_NOTEBOOK_NAME = "31_run_alphafold"
# MODEL_NOTEBOOK_NAME = "31_run_rosetta_ddg"

DATASET_NAME, DATASET_PATH, MODEL_NOTEBOOK_NAME

In [ ]:
TASK_COUNT = 4182

TASK_COUNT

## Workspace

In [ ]:
present_files = []
missing_files = []
for i in tqdm(range(1, TASK_COUNT + 1)):
    path = NOTEBOOK_DIR.parent.joinpath(
        MODEL_NOTEBOOK_NAME, DATASET_NAME, f"shard-{i}-of-{TASK_COUNT}.parquet"
    )
    if i == 1098:
        print(path, path.is_file())
    if path.is_file():
        present_files.append((i, path))
    else:
        missing_files.append((i, path))

In [ ]:
len(present_files), len(missing_files)

| model | finished | running |
| - | - | - |
| proteinsolver | 4166 | 1 |
| protbert | 3474 | 708 |
| alphafold | 1346 | 2836 |
| rosetta | 1608 | 2574 |

In [ ]:
# ",".join([str(m[0]) for m in missing_files])

In [ ]:
pfile = pq.ParquetFile(DATASET_PATH)

assert TASK_COUNT == pfile.num_row_groups

In [ ]:
missing_uniprots = set()
missing_files_for_wt = []

for task_id, path in tqdm(missing_files):
    uniprot_id = (
        pfile.read_row_group(task_id - 1, columns=["protein_id"])
        .to_pandas()["protein_id"]
        .item()
    )
    if uniprot_id in missing_uniprots:
        continue
    missing_files_for_wt.append((task_id, path))
    missing_uniprots.add(uniprot_id)

In [ ]:
len(missing_files_for_wt)

In [ ]:
display(input_df.head(2))
print(len(input_df))